# RePlay Tutorial
This notebook is designed to familiarize with the use of RePlay library, including:
- creating SparkSession or passing your own session to RePlay
- data preprocessing
- dataset users and items re-indexing
- data splitting
- model training and inference
- model optimization
- model saving and loading
- models comparison

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config Completer.use_jedi = False

In [3]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)

In [4]:
import pandas as pd

from pyspark.sql import SparkSession

from replay.data_preparator import DataPreparator, Indexer
from replay.metrics import Coverage, HitRate, NDCG, MAP
from replay.experiment import Experiment
from replay.model_handler import save, load, save_indexer, load_indexer
from replay.models import ALSWrap, ItemKNN, SLIM
from replay.session_handler import get_spark_session, State 
from replay.splitters import UserSplitter
from replay.utils import convert2spark, get_log_info

In [5]:
K = 5
SEED=1234

## Managing SparkSession

RePlay uses Spark as a backend, and thus `SparkSession` should be created before RePlay running. Depends on your needs, you can choose, what to do about `SparkSession`.

- Option 1: use default RePlay `SparkSession`
- You can pass you own session to RePlay. Class `State` stores current session. Here you also have two options: 
    - Option 2: call `get_spark_session` to use default RePlay `SparkSession` with the custom driver memory and number of partitions 
    - Option 3: create `SparkSession` from scratch


### Option 1: use default RePlay's SparkSession
It is the simplest and recommended way for the local execution mode. RePlay will get existing SparkSession or create the new one with default configuration.  Default session parameters are stated in `replay/utils/session_handler.py` file. The driver memory volume and number of partitions depends on available RAM and number of cores respectively.

You could initiate default session creation explicitly, e.g. to preprocess spark DataFrames, get link to SparkUI and set logging level, but if you do not create it by yourself, the session will be created by RePlay anyway.

In [6]:
spark = State().session
spark.sparkContext.setLogLevel('ERROR')
spark

23/09/19 16:00:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
23/09/19 16:00:23 WARN DependencyUtils: Local jar /app/notebooks/test_notebooks/jars/replay_2.12-0.1_spark_3.1.jar does not exist, skipping.
23/09/19 16:00:23 INFO SparkContext: Running Spark version 3.1.3
23/09/19 16:00:23 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/09/19 16:00:23 INFO ResourceUtils: ==============================================================
23/09/19 16:00:23 INFO ResourceUtils: No custom resources configured for spark.driver.
23/09/19 16:00:23 INFO ResourceUtils: ==============================================================
23/09/19 16:00:23 INFO SparkContext: Submitted application: pyspark-shell
23/09/19 16

In [7]:
def print_config_param(session, conf_name):
    # get current spark session configuration:
    conf = session.sparkContext.getConf().getAll()
    # get num partitions
    print(f'{conf_name}: {dict(conf)[conf_name]}')

In [8]:
print_config_param(spark, 'spark.sql.shuffle.partitions')

spark.sql.shuffle.partitions: 168


### Option 2:  Call `get_spark_session`  function to customize driver memory (spark.driver.memory) or number of partitions (spark.sql.shuffle.partitions) and use the default RePlay settings for other configuration parameters.
We will specify 16 partitions and 4Gb driver memory for example. Pass created session to RePlay `State`.

In [9]:
spark.stop()
session = get_spark_session(spark_memory=4, shuffle_partitions=16)
spark = State(session).session

In [10]:
print_config_param(spark, 'spark.sql.shuffle.partitions')

spark.sql.shuffle.partitions: 16


### Option 3: Create your own session
Pass created session to RePlay `State`.

In [11]:
spark.stop()
session = (
        SparkSession.builder.config("spark.driver.memory", "8g")
        .config("spark.sql.shuffle.partitions", "50")
        .config("spark.driver.bindAddress", "127.0.0.1")
        .config("spark.driver.host", "localhost")
        .master("local[*]")
        .enableHiveSupport()
        .getOrCreate()
    )
spark = State(session).session
print_config_param(spark, 'spark.sql.shuffle.partitions')

spark.sql.shuffle.partitions: 50


#### Will return to the default session config

In [12]:
spark.stop()
spark = State(get_spark_session()).session
spark.sparkContext.setLogLevel('ERROR')
spark

## 0. Data preprocessing <a name='data-preparator'></a>
We will use MovieLens 1m as an example.

In [9]:
df = pd.read_csv("data/ml1m_ratings.dat", sep="\t", names=["userId", "item_id", "relevance", "timestamp"])
users = pd.read_csv("data/ml1m_users.dat", sep="\t", names=["user_id", "gender", "age", "occupation", "zip_code"])

FileNotFoundError: [Errno 2] No such file or directory: 'data/ml1m_ratings.dat'

In [9]:
df = pd.read_parquet("../../data/foursquare/interactions.parquet")

In [10]:
df.head(2)

,userId,venueCategoryId,timestamp,relevant
0,470,4bf58dd8d48988d127951735,1.333462e+09,1
1,979,4bf58dd8d48988d1df941735,1.333462e+09,1


In [15]:
users.head(2)

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072


### 0.1. DataPreparator

An inner data format in RePlay is a spark dataframe. 

Columns with users' and items' identifiers are required for interaction log. Original user and item identifiers should be named as `user_id` and `item_id`. Those identifiers in section [0.3. Indexing](#indexing) will be converted to integer identifiers, which will be named `user_idx`, `item_idx`. Optional columns for interaction matrix are ``relevance`` and interaction ``timestamp``.

DataFrames with user or item features should have column `user_id` or `item_id` respectively.

We implemented DataPreparator class to convert pandas dataframes to spark format and preprocess the data, including renaming/creation of required and optional interaction matrix columns, null check and dates parsing. It is an optional step, if you already have data in Spark DataFrame format, could rename the above mentioned columns, and confident in completeness and quality of the data, skip this step.

In [11]:
preparator = DataPreparator()

#### Interactions log preprocessing

In [12]:
%%time
log = preparator.transform(columns_mapping={'user_id': 'userId',
                                      'item_id': 'venueCategoryId',
                                      'relevance': 'relevant',
                                      'timestamp': 'timestamp'
                                     }, 
                           data=df)

19-Sep-23 16:01:50, replay, INFO: Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.


CPU times: user 170 ms, sys: 24.1 ms, total: 194 ms
Wall time: 6.16 s


In [13]:
log.show(2)

+-------+--------------------+-------------------+---------+
|user_id|             item_id|          timestamp|relevance|
+-------+--------------------+-------------------+---------+
|    470|4bf58dd8d48988d12...|2012-04-03 18:00:09|      1.0|
|    979|4bf58dd8d48988d1d...|2012-04-03 18:00:25|      1.0|
+-------+--------------------+-------------------+---------+
only showing top 2 rows



In [14]:
log.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- relevance: double (nullable = true)



In [15]:
get_log_info(log, user_col='user_id', item_col='item_id')

'total lines: 227428, total users: 1083, total items: 400'

As you see, `userId` was renamed to `user_id` and `timestamp` was converted to `TimestampType`.

#### Feature dataframe preprocessing
To transform feature dataframes you could also use DataPreparator:

In [17]:
user_features = preparator.transform(columns_mapping={'user_id': 'user_id'},
                                     data=users)
user_features.show(2)

NameError: name 'users' is not defined

Using the DataPreparator is optional, you could convert dataFrame to spark with ``convert_to_spark`` from ``replay.utils.spark_utils.py`` and manually rename columns.

In [16]:
# the same result without DataPreparator
convert2spark(users).show(2)

NameError: name 'users' is not defined

### 0.2 Filtering
It is common to filter interactions log by interaction date or rating value or remove items or users with small number of interactions. RePlay offers some filters presented in `replay.preprocessig.filters` module.
We will leave ratings greater than or equal to 3 and remove users with 4 or fewer interactions.

In [22]:
from replay.filters import filter_by_min_count, filter_out_low_ratings

In [26]:
log = filter_out_low_ratings(log, value=5)
get_log_info(log, user_col='user_id', item_col='item_id')

'total lines: 0, total users: 0, total items: 0'

In [27]:
%%time
log = filter_by_min_count(log, num_entries=5, group_by='user_id')
get_log_info(log, user_col='user_id', item_col='item_id')

ZeroDivisionError: division by zero

<a id='indexing'></a>
### 0.3. Indexing

RePlay models require columns with users' and items' identifiers _(ids)_ to be named as `user_idx` and `item_idx`. Those _ids_ should be integers starting at zero without gaps. This is important for models that use sparse matrices and define the matrix size as the biggest seen user and item index. Storing _ids_ as integers also help to reduce memory usage compared to string _ids_.

You should convert user and item _ids_ in interaction's log and feature dataframes. RaPlay offers Indexer class to perform the _ids_ conversion and convert them back after recommendations generation (predict). The Indexer will store label encoders for users and items and allow transforming ids for users and items, which come after the Indexer fit.

In [17]:
indexer = Indexer(user_col='user_id', item_col='item_id')

Take all available user and item ids from log and features and pass them to Indexer. The _ids_ could repeat, the indexes will be ordered by label frequencies, so the most frequent label gets index 0.

In [18]:
%%time
indexer.fit(users=log.select('user_id').unionByName(log.select('user_id').distinct()),
            items=log.select('item_id'))

CPU times: user 51.2 ms, sys: 11.9 ms, total: 63.1 ms
Wall time: 3.28 s


In [19]:
%%time
log_replay = indexer.transform(df=log)
log_replay.show(2)

+--------+--------+-------------------+---------+
|user_idx|item_idx|          timestamp|relevance|
+--------+--------+-------------------+---------+
|     326|     157|2012-04-03 18:00:09|      1.0|
|     403|      33|2012-04-03 18:00:25|      1.0|
+--------+--------+-------------------+---------+
only showing top 2 rows

CPU times: user 75.1 ms, sys: 2.68 ms, total: 77.7 ms
Wall time: 2.01 s


In [29]:
%%time
user_features_replay = indexer.transform(df=user_features)
user_features_replay.show(2)

+--------+------+---+----------+--------+
|user_idx|gender|age|occupation|zip_code|
+--------+------+---+----------+--------+
|    3861|     F|  1|        10|   48067|
|    2301|     M| 56|        16|   70072|
+--------+------+---+----------+--------+
only showing top 2 rows

CPU times: user 36.9 ms, sys: 7.24 ms, total: 44.1 ms
Wall time: 352 ms


### 0.4. Split

RePlay provides you with data splitters to reproduce a validation schemas widely-used in recommender systems. Splitters return cached dataframes to compute them once and re-use for models training, inference and metrics calculation.

`UserSplitter` takes ``item_test_size`` items for ``user_test_size`` user to the test dataset.

In [20]:
%%time
splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=K,
    user_test_size=500,
    seed=SEED,
    shuffle=True
)
train, test = splitter.split(log_replay)
print(train.count(), test.count())

224928 2500
CPU times: user 46.1 ms, sys: 10.8 ms, total: 56.9 ms
Wall time: 10.1 s


In [21]:
test.is_cached

True

## 1. Models training

#### SLIM

In [22]:
slim = ItemKNN()

In [23]:
%%time
slim.fit(log=train)

CPU times: user 64.2 ms, sys: 15.9 ms, total: 80.1 ms
Wall time: 11.2 s


In [24]:
%%time

recs = slim.predict(
    k=K,
    users=test.select('user_idx').distinct(),
    log=train,
    filter_seen_items=True
)

CPU times: user 52.6 ms, sys: 24.3 ms, total: 76.9 ms
Wall time: 9.33 s


In [25]:
recs.show(2)

+--------+--------+------------------+
|user_idx|item_idx|         relevance|
+--------+--------+------------------+
|     282|      10|1446.9782879185582|
|     282|      14| 690.9596616351608|
+--------+--------+------------------+
only showing top 2 rows



## 2. Models evaluation

RePlay implements some popular recommenders' quality metrics. Use pure metrics or calculate a set of chosen metrics and compare models with the ``Experiment`` class.

In [26]:
metrics = Experiment(test, {NDCG(): K,
                            MAP() : K,
                            HitRate(): [1, K],
                            Coverage(train): K
                           })

In [27]:
%%time
metrics.add_result("SLIM", recs)
metrics.results

CPU times: user 209 ms, sys: 81.6 ms, total: 291 ms
Wall time: 15.4 s


,Coverage@5,HitRate@1,HitRate@5,MAP@5,NDCG@5
SLIM,0.18,0.004,0.032,0.00304,0.007292


## 3. Hyperparameters optimization

#### 3.1 Search

In [28]:
# data split for hyperparameters optimization
train_opt, val_opt = splitter.split(train)

In [29]:
%%time
best_params = slim.optimize(train_opt, val_opt, criterion=NDCG(), k=K, budget=15)

[I 2023-09-19 16:03:56,894] A new study created in memory with name: no-name-79bd8d9d-6520-4f66-b223-52ef7d5008de
[I 2023-09-19 16:04:18,088] Trial 0 finished with value: 0.0036188326195130452 and parameters: {'num_neighbours': 10, 'shrink': 0, 'weighting': None}. Best is trial 0 with value: 0.0036188326195130452.
[I 2023-09-19 16:04:34,883] Trial 1 finished with value: 0.005302278257183729 and parameters: {'num_neighbours': 36, 'shrink': 63, 'weighting': None}. Best is trial 1 with value: 0.005302278257183729.
[I 2023-09-19 16:04:50,608] Trial 2 finished with value: 0.0040944696867979006 and parameters: {'num_neighbours': 14, 'shrink': 58, 'weighting': 'bm25'}. Best is trial 1 with value: 0.005302278257183729.
[I 2023-09-19 16:05:06,264] Trial 3 finished with value: 0.005604317493639339 and parameters: {'num_neighbours': 41, 'shrink': 4, 'weighting': 'bm25'}. Best is trial 3 with value: 0.005604317493639339.
[I 2023-09-19 16:05:21,973] Trial 4 finished with value: 0.004911426696532679

CPU times: user 2.63 s, sys: 773 ms, total: 3.41 s
Wall time: 4min 3s


In [30]:
best_params

{'num_neighbours': 71, 'shrink': 76, 'weighting': 'tf_idf'}

#### 3.2 Compare with previous

In [31]:
def fit_predict_evaluate(model, experiment, name):
    model.fit(log=train)

    recs = model.predict(
        k=K,
        users=test.select('user_idx').distinct(),
        log=train,
        filter_seen_items=True
    )

    experiment.add_result(name, recs)
    return recs

In [34]:
%%time
recs = fit_predict_evaluate(ItemKNN(**best_params), metrics, 'SLIM_optimized')
recs.cache() #caching for further processing
metrics.results.sort_values('NDCG@5', ascending=False)

CPU times: user 347 ms, sys: 124 ms, total: 472 ms
Wall time: 35.2 s


,Coverage@5,HitRate@1,HitRate@5,MAP@5,NDCG@5
SLIM_optimized,0.175,0.010,0.056,0.005287,0.012968
SLIM,0.180,0.004,0.032,0.003040,0.007292


The optimized model was better on the validation dataset, but shows comparable quality on test (better by HitRate@5 and worse by the other quality metrics). 

## 4. Getting final recommendations 

### Return to original user and item identifiers

In [43]:
%%time
recs = indexer.inverse_transform(recs)
recs.show(2)

+-------+-------+------------------+
|user_id|item_id|         relevance|
+-------+-------+------------------+
|   5107|    527| 1.046002020356746|
|   5107|   2599|0.9492305434804991|
+-------+-------+------------------+
only showing top 2 rows

CPU times: user 754 ms, sys: 296 ms, total: 1.05 s
Wall time: 6.67 s


### Convert to pandas or save

In [44]:
recs_pd = recs.toPandas()
recs_pd.head(2)

,user_id,item_id,relevance
0,5107,527,1.046002
1,5107,2599,0.949231


In [45]:
%%time
recs.write.parquet(path='./slim_recs.parquet', mode='overwrite')

CPU times: user 4.62 ms, sys: 4.24 ms, total: 8.86 ms
Wall time: 2.75 s



[Stage 2751:==================>                                    (8 + 8) / 24]



## 4. Save and load

RePlay allows saving and loading fitted models with `save` and `load` functions of `model_handler` module. Model is saved as a folder with all necessary parameters and data.

In [46]:
%%time
save_indexer(indexer, './indexer_ml1')
indexer = load_indexer('./indexer_ml1')


[Stage 2762:>                                                       (0 + 1) / 1]



CPU times: user 715 ms, sys: 273 ms, total: 988 ms
Wall time: 4.57 s


In [47]:
%%time
save(slim, path='./slim_best_params')
slim_loaded = load('./slim_best_params')

CPU times: user 66.1 ms, sys: 59 ms, total: 125 ms
Wall time: 42.4 s


In [48]:
slim_loaded.beta, slim_loaded.lambda_

(0.061429815496712774, 0.02613996164121192)

In [49]:
%%time
pred_from_loaded = slim_loaded.predict(k=K,
    users=test.select('user_idx').distinct(),
    log=train,
    filter_seen_items=True)
pred_from_loaded.show(2)

13-Jul-22 14:54:38, replay, INFO: This model can't predict cold items, they will be ignored


+--------+--------+------------------+
|user_idx|item_idx|         relevance|
+--------+--------+------------------+
|     282|      15|1.0362708717674016|
|     282|      61|0.9560245729412186|
+--------+--------+------------------+
only showing top 2 rows

CPU times: user 66.2 ms, sys: 33.3 ms, total: 99.5 ms
Wall time: 10.4 s


In [50]:
%%time
recs = indexer.inverse_transform(pred_from_loaded)
recs.show(2)

+-------+-------+------------------+
|user_id|item_id|         relevance|
+-------+-------+------------------+
|   5107|    527|1.0362708717674016|
|   5107|   2599|0.9560245729412186|
+-------+-------+------------------+
only showing top 2 rows

CPU times: user 886 ms, sys: 341 ms, total: 1.23 s
Wall time: 7.32 s


## 5. Other RePlay models

#### ALS
Commonly-used matrix factorization algorithm.

In [51]:
%%time
recs = fit_predict_evaluate(ALSWrap(rank=100, seed=SEED), metrics, 'ALS')
metrics.results.sort_values('NDCG@5', ascending=False)

13-Jul-22 14:55:47, replay, INFO: This model can't predict cold users, they will be ignored
13-Jul-22 14:55:47, replay, INFO: This model can't predict cold items, they will be ignored


CPU times: user 746 ms, sys: 450 ms, total: 1.2 s
Wall time: 3min 11s


,Coverage@5,HitRate@1,HitRate@5,MAP@5,NDCG@5
SLIM,0.151916,0.220,0.556,0.100060,0.172416
SLIM_optimized,0.141163,0.214,0.568,0.099273,0.172007
ALS,0.196305,0.202,0.550,0.086360,0.156045


#### ItemKNN
Commonly-used item-based recommender

In [52]:
%%time
recs = fit_predict_evaluate(ItemKNN(num_neighbours=100), metrics, 'ItemKNN')
metrics.results.sort_values('NDCG@5', ascending=False)

13-Jul-22 15:00:29, replay, INFO: This model can't predict cold items, they will be ignored


CPU times: user 373 ms, sys: 413 ms, total: 786 ms
Wall time: 3min 3s


,Coverage@5,HitRate@1,HitRate@5,MAP@5,NDCG@5
SLIM,0.151916,0.220,0.556,0.100060,0.172416
SLIM_optimized,0.141163,0.214,0.568,0.099273,0.172007
ALS,0.196305,0.202,0.550,0.086360,0.156045
ItemKNN,0.050731,0.168,0.390,0.063387,0.113365


## 6 Compare RePlay models with others
To easily evaluate recommendations obtained from other sources, read and pass these recommendations to ``Experiment``

In [53]:
import pyspark.sql.functions as sf

In [54]:
metrics.add_result("my_model", recs.withColumn("relevance", sf.rand()))
metrics.results.sort_values("NDCG@5", ascending=False)

,Coverage@5,HitRate@1,HitRate@5,MAP@5,NDCG@5
SLIM,0.151916,0.220,0.556,0.100060,0.172416
SLIM_optimized,0.141163,0.214,0.568,0.099273,0.172007
ALS,0.196305,0.202,0.550,0.086360,0.156045
ItemKNN,0.050731,0.168,0.390,0.063387,0.113365
my_model,0.050731,0.118,0.390,0.055720,0.104047
